O objetivo do código neste arquivo é verificar a estacionariedade nas séries e criar um arquivo com as séries estacionarizadas

In [10]:
import pandas as pd
import numpy as np
import utils
import sktime.param_est.stationarity as statio
from sktime.transformations.series.impute import Imputer

In [12]:
df = pd.read_csv("dados/dados_crus2003.csv", sep=';', thousands=".", decimal=",")
df = utils.index_to_period(df)
df

,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,...,result_primario,el_nino,epu_br,epu_usa,ind_conf_cons,desemprego,rendimento,crb_cmdt,crb_food,crb_metal
month,,,,,,,,,,,,,,,,,,,,,
2003-01,2.25,1.98,25.145161,94977783,391237088,696054502,810272451,10941.09,3.4384,100.585,...,48763.52,0.9,88.150920,185.591626,103.0,14.068293,NaN,250.14,239.98,203.34
2003-02,1.57,2.19,25.821429,93602937,393976058,706788435,815745074,10280.61,3.5908,103.518,...,51066.28,0.6,120.281038,161.759767,103.5,14.570732,NaN,250.13,233.46,206.19
2003-03,1.23,1.14,26.500000,89822584,388142457,710712013,820859498,11273.63,3.4469,98.302,...,51462.26,0.4,133.688711,224.647203,101.9,15.198780,182585.2573,248.06,234.18,202.43
2003-04,0.97,1.14,26.500000,87410599,385467653,712800120,823489095,12556.70,3.1187,88.069,...,55840.39,0.0,96.791932,176.579867,107.7,15.575610,178958.1301,247.57,235.03,204.02
2003-05,0.61,0.85,26.500000,86231763,388490690,721321334,833412205,13421.60,2.9557,86.559,...,57077.43,-0.3,85.230528,122.309831,112.0,16.203659,177284.6602,250.48,238.04,206.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,0.23,0.51,13.750000,577826684,5322265166,10022339920,11110115291,108335.07,4.9828,53.920,...,39023.18,0.5,137.801645,205.000000,122.2,8.300000,282700.0000,541.45,532.58,987.00
2023-06,-0.08,0.04,13.750000,588295220,5423979831,10218926744,11238752433,118087.00,4.8516,53.000,...,-24270.03,0.8,169.261123,179.000000,125.3,8.000000,285243.0000,547.83,543.66,992.69
2023-07,0.12,-0.07,13.750000,617150371,5502967358,10359936160,11332427031,121942.98,4.8008,53.410,...,-80519.87,1.1,133.688711,133.000000,124.5,7.900000,287672.0000,563.85,567.31,1021.05


In [15]:
# Imputa valores faltantes de rendimento
imputer = Imputer()
df['rendimento'] = imputer.fit_transform(df['rendimento'])
df['rendimento']

C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\transformations\series\impute.py:349: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y=self._X[col].ffill().fillna(method="backfill"),


month
2003-01    178968.971030
2003-02    179404.398363
2003-03    182585.257300
2003-04    178958.130100
2003-05    177284.660200
               ...      
2023-05    282700.000000
2023-06    285243.000000
2023-07    287672.000000
2023-08    289421.000000
2023-09    292952.000000
Freq: M, Name: rendimento, Length: 249, dtype: float64

In [18]:
df.isna().any()

ipca                    False
ipca_15                 False
selic                   False
m1                      False
m2                      False
m3                      False
m4                      False
ibovespa                False
cambio                  False
cambio_real             False
ind_varejo              False
ind_varejo_ampl         False
ind_industria           False
ibc_br                  False
capacidade_industria    False
salario_minimo          False
energia_total           False
energia_residencial     False
energia_industrial      False
energia_comercial       False
energia_outros          False
oil_usa                 False
oil_eu                  False
pib                     False
bens_capital            False
bens_int                False
bens_cons_d             False
bens_cons_nd            False
prod_veiculos           False
prod_caminhoes          False
prod_onibus             False
result_primario         False
el_nino                 False
epu_br    

In [20]:
# Faz o teste Kwiatkowski-Phillips-Schmidt-Shin (KPSS) para estacionariedade
first_results = utils.test_for_stationarity_kpss(df)
utils.print_stationary_series(first_results)

ipca
ipca_15
oil_usa
oil_eu
bens_capital
prod_veiculos
prod_caminhoes
el_nino
desemprego


C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of 

In [23]:
# Remove séries que já são estacionárias
stationary_series = [
    'ipca',
    'ipca_15',
    'oil_usa',
    'oil_eu',
    'bens_capital',
    'prod_veiculos',
    'prod_caminhoes',
    'el_nino',
    'desemprego',
]
# Tira a primeira diferença e tira valores nulos (jan/2003)
diff1_df = df.drop(columns=stationary_series).diff().dropna()
diff1_df


,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,ind_varejo,ind_varejo_ampl,...,bens_cons_nd,prod_onibus,result_primario,epu_br,epu_usa,ind_conf_cons,rendimento,crb_cmdt,crb_food,crb_metal
month,,,,,,,,,,,,,,,,,,,,,
2003-02,0.676267,-1374846.0,2738970.0,10733933.0,5472623.0,-660.48,0.1524,2.933,-0.30700,-0.35824,...,-3.60457,46.0,2302.76,32.130118,-23.831859,0.5,435.427333,-0.01,-6.52,2.85
2003-03,0.678571,-3780353.0,-5833601.0,3923578.0,5114424.0,993.02,-0.1439,-5.216,0.87713,0.30706,...,3.50633,29.0,395.98,13.407673,62.887436,-1.6,3180.858937,-2.07,0.72,-3.76
2003-04,0.000000,-2411985.0,-2674804.0,2088107.0,2629597.0,1283.07,-0.3282,-10.233,0.39471,0.56295,...,0.02111,162.0,4378.13,-36.896779,-48.067337,5.8,-3627.127200,-0.49,0.85,1.59
2003-05,0.000000,-1178836.0,3023037.0,8521214.0,9923110.0,864.90,-0.1630,-1.510,0.70170,0.81882,...,4.82833,127.0,1237.04,-11.561404,-54.270036,4.3,-1673.469900,2.91,3.01,2.52
2003-06,-0.183333,2234552.0,-2152663.0,10208771.0,6985494.0,-449.02,-0.0725,-0.308,-1.27183,-1.22824,...,-3.41149,-93.0,-2498.83,-28.352196,-11.614289,4.8,-1652.838600,-1.42,-3.46,-0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,0.000000,-23521679.0,66148719.0,17897571.0,26468887.0,3903.44,-0.0369,-0.740,1.75850,5.07920,...,14.89350,319.0,-17179.69,-66.227511,40.000000,-2.9,658.000000,-6.00,4.55,-69.98
2023-06,0.000000,10468536.0,101714665.0,196586824.0,128637142.0,9751.93,-0.1312,-0.920,-2.28240,-0.82830,...,-1.93460,2.0,-63293.21,31.459478,-26.000000,3.1,2543.000000,6.38,11.08,5.69
2023-07,0.000000,28855151.0,78987527.0,141009416.0,93674598.0,3855.98,-0.0508,0.410,3.11000,-0.25900,...,7.03730,-46.0,-56249.84,-35.572412,-46.000000,-0.8,2429.000000,16.02,23.65,28.36


In [24]:
diff_results = utils.test_for_stationarity_kpss(diff1_df)
utils.print_stationary_series(diff_results)

selic
m1
ibovespa
cambio
cambio_real
ind_varejo
ind_varejo_ampl
ind_industria
ibc_br
capacidade_industria
salario_minimo
energia_total
energia_residencial
energia_industrial
energia_comercial
energia_outros
bens_int
bens_cons_d
bens_cons_nd
prod_onibus
result_primario
epu_br
epu_usa
ind_conf_cons
rendimento
crb_cmdt
crb_food
crb_metal


C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of 

In [25]:
# Remove séries estacionárias de novo
stationary_series2 = [
    'selic',
    'm1',
    'ibovespa',
    'cambio',
    'cambio_real',
    'ind_varejo',
    'ind_varejo_ampl',
    'ind_industria',
    'ibc_br',
    'capacidade_industria',
    'salario_minimo',
    'energia_total',
    'energia_residencial',
    'energia_industrial',
    'energia_comercial',
    'energia_outros',
    'bens_int',
    'bens_cons_d',
    'bens_cons_nd',
    'prod_onibus',
    'result_primario',
    'epu_br',
    'epu_usa',
    'ind_conf_cons',
    'rendimento',
    'crb_cmdt',
    'crb_food',
    'crb_metal',
]
# Tira a segunda diferença e tira valores nulos (fev/2003)
diff2_df = diff1_df.drop(columns=stationary_series2).diff().dropna()
diff2_df

,m2,m3,m4,pib
month,,,,
2003-03,-8572571.0,-6810355.0,-358199.0,1796.7
2003-04,3158797.0,-1835471.0,-2484827.0,-1085.4
2003-05,5697841.0,6433107.0,7293513.0,-4111.7
2003-06,-5175700.0,1687557.0,-2937616.0,-2004.0
2003-07,4628574.0,7366156.0,9313824.0,3267.9
...,...,...,...,...
2023-05,37144204.0,-60304921.0,-53107060.0,-8420.6
2023-06,35565946.0,178689253.0,102168255.0,-3658.6
2023-07,-22727138.0,-55577408.0,-34962544.0,1475.6


In [26]:
diff2_results = utils.test_for_stationarity_kpss(diff2_df)
utils.print_stationary_series(diff2_results)

m2
m3
m4
pib


C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of 

In [30]:
# Cria DataFrame apenas com séries estacionárias
new_df = df.copy()
for col in diff1_df:
    new_df[col] = diff1_df[col]
for col in diff2_df:
    new_df[col] = diff2_df[col]
new_df = new_df.dropna()
new_df

,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,...,result_primario,el_nino,epu_br,epu_usa,ind_conf_cons,desemprego,rendimento,crb_cmdt,crb_food,crb_metal
month,,,,,,,,,,,,,,,,,,,,,
2003-03,1.23,1.14,0.678571,-3780353.0,-8572571.0,-6810355.0,-358199.0,993.02,-0.1439,-5.216,...,395.98,0.4,13.407673,62.887436,-1.6,15.198780,3180.858937,-2.07,0.72,-3.76
2003-04,0.97,1.14,0.000000,-2411985.0,3158797.0,-1835471.0,-2484827.0,1283.07,-0.3282,-10.233,...,4378.13,0.0,-36.896779,-48.067337,5.8,15.575610,-3627.127200,-0.49,0.85,1.59
2003-05,0.61,0.85,0.000000,-1178836.0,5697841.0,6433107.0,7293513.0,864.90,-0.1630,-1.510,...,1237.04,-0.3,-11.561404,-54.270036,4.3,16.203659,-1673.469900,2.91,3.01,2.52
2003-06,-0.15,0.22,-0.183333,2234552.0,-5175700.0,1687557.0,-2937616.0,-449.02,-0.0725,-0.308,...,-2498.83,-0.2,-28.352196,-11.614289,4.8,16.329268,-1652.838600,-1.42,-3.46,-0.18
2003-07,0.20,-0.18,-0.703763,-1416187.0,4628574.0,7366156.0,9313824.0,599.15,-0.0034,-0.296,...,629.86,0.1,32.737046,-6.947295,-5.3,16.078049,-25.622900,2.08,-2.61,9.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,0.23,0.51,0.000000,-23521679.0,37144204.0,-60304921.0,-53107060.0,3903.44,-0.0369,-0.740,...,-17179.69,0.5,-66.227511,40.000000,-2.9,8.300000,658.000000,-6.00,4.55,-69.98
2023-06,-0.08,0.04,0.000000,10468536.0,35565946.0,178689253.0,102168255.0,9751.93,-0.1312,-0.920,...,-63293.21,0.8,31.459478,-26.000000,3.1,8.000000,2543.000000,6.38,11.08,5.69
2023-07,0.12,-0.07,0.000000,28855151.0,-22727138.0,-55577408.0,-34962544.0,3855.98,-0.0508,0.410,...,-56249.84,1.1,-35.572412,-46.000000,-0.8,7.900000,2429.000000,16.02,23.65,28.36


In [31]:
# Só garantindo
new_df_results = utils.test_for_stationarity_kpss(new_df)
utils.print_stationary_series(new_df_results)

ipca
ipca_15
selic
m1
m2
m3
m4
ibovespa
cambio
cambio_real
ind_varejo
ind_varejo_ampl
ind_industria
ibc_br
capacidade_industria
salario_minimo
energia_total
energia_residencial
energia_industrial
energia_comercial
energia_outros
oil_usa
oil_eu
pib
bens_capital
bens_int
bens_cons_d
bens_cons_nd
prod_veiculos
prod_caminhoes
prod_onibus
result_primario
el_nino
epu_br
epu_usa
ind_conf_cons
desemprego
rendimento
crb_cmdt
crb_food
crb_metal


C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of 

In [32]:
new_df.to_csv("dados/dados_transformados2003.csv", sep=';', decimal=',')

In [35]:
# Cria arquivo com dados apenas de 2004
rows_to_drop = [
    '2003-03',
    '2003-04',
    '2003-05',
    '2003-06',
    '2003-07',
    '2003-08',
    '2003-09',
    '2003-10',
    '2003-11',
    '2003-12',
]
new_df = new_df.drop(rows_to_drop)
new_df.to_csv("dados/dados_transformados.csv", sep=';', decimal=',')